In [ ]:
import sys
sys.path.append('../src')

import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from data_loader import WildFireDataLoader
from model_loader import ModelLoader
import pandas as pd

# 1. ИНИЦИАЛИЗАЦИЯ
print("=== ДИАГНОСТИКА МОДЕЛЕЙ ===")

data_loader = WildFireDataLoader("../data/raw")
data_dict = data_loader.load_all_data()
test_df = data_dict['test']

model_loader = ModelLoader("cuda")
model_loader.load_all_models()

# 2. ПРОВЕРКА СООТВЕТСТВИЯ МЕТОК
print("\n=== ПРОВЕРКА МЕТОК ===")
print(f"Наш mapping: {data_loader.class_mapping}")

# Проверка id2label у моделей
for model_key in ['base', 'finetuned']:
    model = model_loader.models[model_key]
    print(f"\nМодель '{model_key}':")
    print(f"  id2label: {model.config.id2label}")
    print(f"  Количество классов: {model.config.num_labels}")

# 3. ТЕСТ НА КОНКРЕТНЫХ ИЗОБРАЖЕНИЯХ
print("\n=== ТЕСТ НА КОНКРЕТНЫХ ИЗОБРАЖЕНИЯХ ===")

# Возьмем по 2 примера каждого класса
fire_samples = test_df[test_df['label'] == 0].head(2)
no_fire_samples = test_df[test_df['label'] == 1].head(2)

results = []

for idx, row in pd.concat([fire_samples, no_fire_samples]).iterrows():
    img = Image.open(row['image_path']).convert('RGB')
    true_label = row['label']
    true_class = 'fire' if true_label == 0 else 'no_fire'
    
    sample_result = {
        'image_path': row['image_path'],
        'true_label': true_label,
        'true_class': true_class
    }
    
    for model_key in ['base', 'finetuned']:
        model, processor = model_loader.models[model_key], model_loader.processors[model_key]
        
        # Предобработка
        inputs = processor(img, return_tensors="pt").to(model_loader.device)
        
        # Инференс
        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]
            pred = torch.argmax(outputs.logits, dim=-1).item()
        
        sample_result[f'{model_key}_pred'] = pred
        sample_result[f'{model_key}_prob_fire'] = probs[0].item()
        sample_result[f'{model_key}_prob_no_fire'] = probs[1].item()
        sample_result[f'{model_key}_correct'] = pred == true_label
    
    results.append(sample_result)

# Вывод результатов
print("\nРезультаты на отдельных примерах:")
for res in results:
    print(f"\nИзображение: {res['image_path'].split('/')[-1]}")
    print(f"Истинный класс: {res['true_class']} (label={res['true_label']})")
    for model_key in ['base', 'finetuned']:
        print(f"  {model_key}: pred={res[f'{model_key}_pred']} "
              f"(fire_prob={res[f'{model_key}_prob_fire']:.3f}, "
              f"no_fire_prob={res[f'{model_key}_prob_no_fire']:.3f}) "
              f"{'✓' if res[f'{model_key}_correct'] else '✗'}")

# 4. АНАЛИЗ РАСПРЕДЕЛЕНИЯ ВЕРОЯТНОСТЕЙ
print("\n=== АНАЛИЗ РАСПРЕДЕЛЕНИЯ ВЕРОЯТНОСТЕЙ ===")

# Проанализируем 100 случайных изображений
sample_df = test_df.sample(min(100, len(test_df)), random_state=42)

fire_probs_base = []
fire_probs_finetuned = []

for _, row in sample_df.iterrows():
    img = Image.open(row['image_path']).convert('RGB')
    
    for model_key, probs_list in [('base', fire_probs_base), 
                                   ('finetuned', fire_probs_finetuned)]:
        model, processor = model_loader.models[model_key], model_loader.processors[model_key]
        inputs = processor(img, return_tensors="pt").to(model_loader.device)
        
        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]
            probs_list.append(probs[0].item())  # Вероятность класса "fire"

# Визуализация распределений
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].hist(fire_probs_base, bins=20, alpha=0.7, color='red')
axes[0].set_title('Распределение вероятности "fire" (base модель)')
axes[0].set_xlabel('Вероятность')
axes[0].set_ylabel('Частота')
axes[0].axvline(0.5, color='black', linestyle='--', label='Порог 0.5')
axes[0].legend()

axes[1].hist(fire_probs_finetuned, bins=20, alpha=0.7, color='blue')
axes[1].set_title('Распределение вероятности "fire" (finetuned модель)')
axes[1].set_xlabel('Вероятность')
axes[1].set_ylabel('Частота')
axes[1].axvline(0.5, color='black', linestyle='--', label='Порог 0.5')
axes[1].legend()

plt.tight_layout()
plt.savefig('../outputs/figures/probability_distribution.png', dpi=150)
plt.show()

# Статистика
print(f"\nСтатистика вероятности 'fire' (base):")
print(f"  Среднее: {np.mean(fire_probs_base):.3f}")
print(f"  Медиана: {np.median(fire_probs_base):.3f}")
print(f"  Минимум: {np.min(fire_probs_base):.3f}")
print(f"  Максимум: {np.max(fire_probs_base):.3f}")
print(f"  Доля >0.5: {np.mean(np.array(fire_probs_base) > 0.5):.3f}")

print(f"\nСтатистика вероятности 'fire' (finetuned):")
print(f"  Среднее: {np.mean(fire_probs_finetuned):.3f}")
print(f"  Медиана: {np.median(fire_probs_finetuned):.3f}")
print(f"  Минимум: {np.min(fire_probs_finetuned):.3f}")
print(f"  Максимум: {np.max(fire_probs_finetuned):.3f}")
print(f"  Доля >0.5: {np.mean(np.array(fire_probs_finetuned) > 0.5):.3f}")

# 5. ПРОВЕРКА ИНВЕРСИИ МЕТОК
print("\n=== ПРОВЕРКА ИНВЕРСИИ МЕТОК ===")
print("Попробуем инвертировать предсказания (предположим, что модель обучена с обратными метками)")

inverted_correct_base = []
inverted_correct_finetuned = []

for _, row in sample_df.iterrows():
    true_label = row['label']
    
    for model_key, correct_list in [('base', inverted_correct_base), 
                                     ('finetuned', inverted_correct_finetuned)]:
        pred = 1 - res[f'{model_key}_pred']  # Инвертируем предсказание
        correct_list.append(pred == true_label)

print(f"Точность с инверсией (base): {np.mean(inverted_correct_base):.3f}")
print(f"Точность с инверсией (finetuned): {np.mean(inverted_correct_finetuned):.3f}")

# 6. ВЫВОДЫ И РЕКОМЕНДАЦИИ
print("\n=== ВЫВОДЫ ===")
print("1. Если модели всегда предсказывают 'fire' (вероятность >0.5 для всех изображений):")
print("   - Модели могут быть смещены из-за дисбаланса в обучающих данных")
print("   - Возможно, требуется калибровка порога")

print("\n2. Если точность с инверсией высокая:")
print("   - Модели обучены с противоположными метками (0=no_fire, 1=fire)")
print("   - Нужно поменять метки в нашем коде")

print("\n3. Если распределение вероятностей равномерное:")
print("   - Модели не обучались должным образом")
print("   - Нужно рассмотреть другие модели или дообучение")

print("\n=== ДАЛЬНЕЙШИЕ ДЕЙСТВИЯ ===")
print("1. Проверьте id2label моделей на Hugging Face")
print("2. Посмотрите примеры использования моделей на страницах:")
print("   - https://huggingface.co/Gurveer05/vit-base-patch16-224-in21k-fire-detection")
print("   - https://huggingface.co/EdBianchi/vit-fire-detection")
print("3. Проверьте баланс классов в тестовом наборе")
print("4. Рассмотрите возможность дообучения моделей на вашем датасете")

INFO:data_loader:Загрузка данных...
INFO:data_loader:Поиск данных в: ../data/raw
INFO:data_loader:Структура: {'train_exists': True, 'test_exists': True, 'train_subfolders': ['nowildfire', 'wildfire'], 'test_subfolders': ['nowildfire', 'wildfire'], 'image_formats': {<built-in method lower of str object at 0x73ef18081710>, <built-in method lower of str object at 0x73ef17f73720>, <built-in method lower of str object at 0x73ef18081770>, <built-in method lower of str object at 0x73ef19053780>, <built-in method lower of str object at 0x73ef17f73780>, <built-in method lower of str object at 0x73ef180817a0>, <built-in method lower of str object at 0x73ef180817d0>, <built-in method lower of str object at 0x73ef18081620>, <built-in method lower of str object at 0x73ef18081650>, <built-in method lower of str object at 0x73ef18083690>, <built-in method lower of str object at 0x73ef180816e0>, <built-in method lower of str object at 0x73ef17f736f0>, <built-in method lower of str object at 0x73ef1808

=== ДИАГНОСТИКА МОДЕЛЕЙ ===


INFO:data_loader:Найдено 15750 изображений для класса wildfire
INFO:data_loader:Найдено 14500 изображений для класса nowildfire
INFO:data_loader:Найдено 3480 изображений для класса wildfire
INFO:data_loader:Найдено 2820 изображений для класса nowildfire
INFO:data_loader:Загрузка завершена. Тренировочных -- 30250, тестовых -- 6300
INFO:model_loader:Используется устройство: cuda



=== ПРОВЕРКА МЕТОК ===
Наш mapping: {'wildfire': 0, 'nowildfire': 1}


KeyError: 'base'